In [1]:
import os
os.chdir("..")

In [2]:
import json
from collections import Counter
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import string
from os import listdir
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords as get_stopwords
from nltk.tokenize import word_tokenize
from classes.constants import *
from classes.claim import Claim
from util.claim_util import *
from util.undefined_util import *
from util.evaluate_util import *
from util.wiki_util import *
from typing import List
from random import sample
import random
from util.google_search_util import google_the_claim
import time

In [3]:
train = load_claims_train()

In [4]:
train_no_nei = [t for t in train if t.label != Claim.NEI]

In [5]:
t = train_no_nei[0]

In [6]:
random.seed(42)

In [7]:
train_subset = sample(train_no_nei, 100)

In [8]:
    Counter([t.label for t in train_subset])

Counter({'SUPPORTS': 74, 'REFUTES': 26})

In [9]:
c = train[0]

In [10]:
c

SUPPORTS: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.

In [11]:
claim = c.claim

In [12]:
google_the_claim(claim, pages_to_check=1)

['Nikolaj_Coster-Waldau']

In [13]:
results = []

In [14]:
len(results)

0

In [15]:
results

[]

In [19]:
train_subset

[SUPPORTS: Black Mirror started airing in 2011.,
 SUPPORTS: Lee Majors's birth month is April.,
 REFUTES: Nelson Mandela was Chinese.,
 REFUTES: The United States Congress has 2 billion voting members.,
 SUPPORTS: Linda McCartney was diagnosed with breast cancer in 1995.,
 SUPPORTS: Judy Greer works as an actor.,
 SUPPORTS: Burundi was formerly a European colony ruled by Germans.,
 SUPPORTS: Jane Eyre is a book.,
 REFUTES: John Bradley-West refused to be in the TV series Game of Thrones.,
 SUPPORTS: Gray wolves will eat garbage.,
 REFUTES: Lauren Bacall was alive at the age of 90.,
 REFUTES: The Hobbit is a type of fire hydrant.,
 REFUTES: Allen Iverson was really tall.,
 SUPPORTS: Citadelle Laferrière is in Nord, Haiti.,
 SUPPORTS: Sa Talaiassa is located within the Balearic Islands of Spain.,
 REFUTES: Tupac Shakur was born on September 14th, 1898.,
 SUPPORTS: All About Eve won an award for Best Picture.,
 REFUTES: Atomic Blonde is based on a 2017 graphic novel.,
 SUPPORTS: The RMS 

In [18]:
results

[({'Black_Mirror', 'List_of_Black_Mirror_episodes'},
  set(),
  set(),
  set(),
  set()),
 ({'Lee_Majors'}, set(), set(), set(), set()),
 (set(), set(), {'China–South_Africa_relations'}, set(), set()),
 ({'51st_United_States_Congress', 'United_States_Congress'},
  set(),
  set(),
  set(),
  set()),
 ({'Linda_McCartney'}, set(), set(), set(), set()),
 ({'Judy_Greer'}, set(), set(), set(), set()),
 ({'Burundi', 'History_of_Burundi'}, set(), set(), set(), set()),
 ({'Jane_Eyre'}, set(), set(), set(), set()),
 (set(), set(), set(), set(), set()),
 (set(), {'Wolf'}, set(), set(), set()),
 ({'Lauren_Bacall'}, set(), set(), set(), set()),
 ({'Fire_hydrant'}, set(), set(), set(), set()),
 (set(), {'Allen_Iverson'}, set(), set(), set()),
 ({'Citadelle_La_Ferrière', 'Citadelle_Laferrière'},
  set(),
  set(),
  set(),
  set()),
 ({'Balearic_Islands',
   'Sa_Talaiassa',
   'Sant_Josep_de_sa_Talaia_(municipality)'},
  set(),
  set(),
  set(),
  set()),
 ({'Tupac_Shakur'}, set(), set(), set(), set()

In [17]:
for i, t in enumerate(train_subset[len(results):]):
    while True:
        try:
            first_page_docs = google_the_claim(t.claim, starting_from_page=0)
            time.sleep(1)
            second_page_docs = google_the_claim(t.claim, starting_from_page=1)
            time.sleep(1)
            third_page_docs = google_the_claim(t.claim, starting_from_page=2)
            time.sleep(1)
            fourth_page_docs = google_the_claim(t.claim, starting_from_page=3)
            time.sleep(1)
            fifth_page_docs = google_the_claim(t.claim, starting_from_page=4)
            time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(60 * 60)
            continue
    
        results.append((set(first_page_docs), set(second_page_docs), set(third_page_docs), set(fourth_page_docs), set(fifth_page_docs)))
        print(i, t.claim, set(first_page_docs))
        break

429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DSeven%2BDays%2Bin%2BMay%2Bwas%2Bdirected%2Bby%2Ba%2Bperson.%26start%3D0&q=EgSwPi09GO_WoeYFIhkA8aeDS4rUVdcIcB6yM-SLjgF4KKwi4hNOMgFy
429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DSeven%2BDays%2Bin%2BMay%2Bwas%2Bdirected%2Bby%2Ba%2Bperson.%26start%3D0&q=EgSwPi09GIDzoeYFIhkA8aeDS5XQ_UdJZxZKAIYt8kj-FpCwynw6MgFy
0 Seven Days in May was directed by a person. {'Seven_Days_in_May'}
1 An American actor starred in The Host. {'Greg_Kinnear', 'James_Lipton'}
2 Richard Curtis worked with Sir Lenny Henry. {'Lenny_Henry', 'Richard_Curtis'}
3 Colm Feore is a stage actor. {'Colm_Feore'}
4 Steve Mouzakis has yet to star in any Australian films. {'Steve_Mouzakis'}
5 Bruce Springsteen received an award. {'Bruce_Springsteen'}
6 Coquette Productions produced Dirt. {'Coquette_Productions'}
7 Dragon is unrelat

KeyboardInterrupt: 

In [16]:
def evaluate_documents_sample(train_subset: List[Claim]):
    documents : List[set] = []
    #for claim in tqdm(train_subset):
    for i, claim in enumerate(train_subset):
        documents_found = retrieve_documents(claim.claim, wiki_titles)
        print(i, claim.claim, claim.evidence_document_sets(). documents_found)
        documents.append(documents_found)
    return evaluate_documents_retrieval_full([c.evidence_document_sets() for c in train_subset], documents)

In [18]:
def retrieve_documents(claim: str, wiki_titles: List[str]) -> set:
    top_4 = google_the_claim(claim, limit_to_wiki=True)[:4]
    return set(top_4)

In [17]:
def retrieve_documents(claim: str, wiki_titles: List[str]) -> set:
    print(claim)
    top_4 = google_the_claim(claim, verbose=True)
    return set(top_4)

In [18]:
retrieve_documents(claim, wiki_titles)

Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.


set()

In [21]:
evaluate_documents_sample(train_subset, wiki_titles)

100%|██████████| 100/100 [02:19<00:00,  1.46s/it]


{'average_precision': 0.0,
 'average_recall': 0.0,
 'average_f1_score': 0.0,
 'average_f2_score': 0.0,
 'oracle_accuracy': 0.0}

In [19]:
evaluate_documents_sample(train_subset, wiki_titles)

100%|██████████| 100/100 [02:26<00:00,  1.28s/it]


{'average_precision': 0.235,
 'average_recall': 0.7515,
 'average_f1_score': 0.3405158730158726,
 'average_f2_score': 0.497013888888889,
 'oracle_accuracy': 0.76}